In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import reciprocal
from reciprocal.kspace import KSpace
from reciprocal.canvas import Canvas, choose_color
from reciprocal.lattice import LatticeVectors, Lattice
from reciprocal.kvector import KVectorGroup
from reciprocal.symmetry import Symmetry
from reciprocal.utils import apply_symmetry_operators
from scipy.interpolate import RectBivariateSpline
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.patches import Circle
import matplotlib.cm as cm
from adaptsamp.sampler2d import AdaptiveSampler2D
from scipy.interpolate import CloughTocher2DInterpolator, LinearNDInterpolator
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['image.cmap']='plasma'



In [ ]:
import shapely

In [ ]:
shapely.__version__

In [ ]:
def get_lattice_vectors(shape):
    period = 1.
    if shape == 'square':
        return LatticeVectors.from_lengths_angle(period, period, 90.)
    elif shape == 'rectangle':
        return LatticeVectors.from_lengths_angle(period*1.5, period, 90.)
    elif shape == 'hexagon':
        return LatticeVectors.from_lengths_angle(period, period, 60.)
    elif shape == 'oblique':
        return LatticeVectors.from_lengths_angle(period, period, 75.)
    
def get_symmetry(shape):
    if shape == 'square':
        return 'D4'
    elif shape == 'rectangle':
        return 'D2'
    elif shape == 'hexagon':
        return 'D6'
    elif shape == 'oblique':
        return 'C2'

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    print(lat_shape)
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    canvas.plot_tesselation(rlat)
    constraint = {'type':'n_points','value':32}
    sampling, weighting, int_element, sym_ops = rlat.unit_cell.sample_irreducible(constraint=constraint)
    
    print("integration element: {}".format(int_element))
    print("unique weighting: {}".format(np.unique(weighting)))
    canvas.plot_point_sampling_weighted(sampling, weighting)
    full_area = rlat.unit_cell.area()
    weight_sum = np.sum(int_element*weighting)
    symmetry_multiplier = rlat.unit_cell.symmetry().get_n_symmetry_ops()
    print("integration value check: {}".format(symmetry_multiplier*weight_sum/full_area))
    plt.xlabel('kx')
    plt.ylabel('ky')
    plt.xlim([-rlat.vectors.length1*1.1, rlat.vectors.length1*1.1])
    plt.ylim([-rlat.vectors.length2*1.1, rlat.vectors.length2*1.1])
    plt.title(lat_shape, fontsize=20)

In [ ]:
def lorentz(gamma, x0, x):
    return (1./np.pi) * 0.5*gamma / ( (x-x0)**2 + (0.5*gamma)**2)

def int_lorentz(gamma, x0, x):
    return (1./np.pi)*np.arctan(2*(x-x0)/gamma)

def int_lorentz_radial(gamma, x0, x):
    return (1./np.pi)*(gamma*np.log( 4*(x-x0)**2 + gamma**2)/4.0 + x0 * np.arctan( 2*(x-x0)/gamma))

def cusp_function(rho, width, pos):
    L = lorentz(width, pos, rho)
    return L
    
def int_cusp_function(rho):
    L_int = int_lorentz(0.05, 0.5, rho)
    return L_int + (10./4.)*rho**4 + 10*rho

def int_cusp_function_radial(rho):
    L_int_radial = int_lorentz_radial(0.05, 0.5, rho)
    return L_int_radial + (10./5.)*rho**5 + (10/2.)*rho**2

def radial_cusp_function(kx, ky, kmax):
    kr = np.sqrt( kx**2 + ky**2)/kmax    
    pos = kmax*0.4
    width = kmax/20
    return cusp_function(kr, width, pos)

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    print(lat_shape)
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])
    canvas = Canvas(ax=ax)
    wvl = 1.0*np.pi
    k0 = 2*np.pi/wvl
    kspace = KSpace(wvl, symmetry=get_symmetry(lat_shape), fermi_radius=k0)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    kspace.apply_lattice(rlat)
    canvas.plot_tesselation(rlat)            
    canvas.plot_fermi_circle(kspace)
    #families = kspace.periodic_sampler.sample_bloch_families(restrict_to_sym_cone=True)
    constraint = {'type':'n_points', 'value':6}
    sampling, weighting, int_element, sym_ops = rlat.unit_cell.sample_irreducible(constraint=constraint)    
    max_lengths = rlat.unit_cell._max_lengths_from_constraint(constraint)
    #print(max_lengths)
    #print(max_lengths[0]*max_lengths[1]*0.5)
    #print(rlat.unit_cell.integration_element(constraint))
    #plt.scatter(sampling[:, 0], sampling[:,1], s=300, c='tab:red')
    #print(sampling.shape)
    #f_vals = radial_cusp_function(sampling[:, 0], sampling[:,1], k0)
    #canvas.plot_point_sampling_weighted(sampling, f_vals)
    #print(len(sym_ops))
    #print(len(sym_ops[0]))
    #canvas.plot_point_sampling(sampling)    
    families, all_kv, sym_ops = kspace.periodic_sampler._bloch_fam_from_sample(sampling, 1e-5, False, symmetries=sym_ops)
    #canvas.plot_bloch_families(families)
    all_points = []
    all_values = []    
    for num, fam in families.items():
        #if not num == 1:
        #    continue
        #color = choose_color(num, len(families))
        #print(num, fam.kx, fam.ky)
        kx = fam.kx
        ky = fam.ky          
        f_vals = radial_cusp_function(fam.kx, fam.ky, k0)
        #f_vals = np.ones(kx.size)
        f_vals = np.atleast_2d(f_vals).T
        #print(f_vals.shape)
        kxy = np.vstack([kx, ky]).T
        #print(kxy.shape)
        print_data = np.hstack([kxy, f_vals])
        #print(print_data.shape)
        #print(print_data)
        #canvas.plot_point_sampling_weighted(fam.k[:, :2], f_vals)
        #plt.scatter(fam.kx, fam.ky, s=200, c='k')
        #print(fam.kx, fam.ky, fam.order1, fam.order2, sym_ops[num])
        #for row in range(fam.n_rows):
        #print(sym_ops[num])
        #print(sym_ops[num])
        zeroth_order = np.where(np.logical_and(fam.order1==0, fam.order2==0))
        #print(fam.kx[zeroth_order], fam.ky[zeroth_order], sym_ops[num])
        sym = rlat.unit_cell.symmetry()-sym_ops[num]
        #print(sym)
        #print(fam.k.shape, f_vals.shape)        
        sym_points, sym_vals = sym.apply_symmetry_operators(fam.k, values=f_vals)
        #print(sym_points.shape, sym_vals.shape)
        sym_points, indices = np.unique(sym_points.round(decimals=4), axis=0, return_index=True)
        sym_vals = sym_vals[indices, :]
        #print(fam.n_rows, sym_points.shape[0])
        #print(sym_points)
        all_points.append(sym_points)
        all_values.append(sym_vals)
        #plt.scatter(sym_points[:,0], sym_points[:,1], s=50, c='tab:blue')
        #print(sym)
        #print(sym_points)
        #print(sym_points.shape[0])        
        #color = choose_color(num, len(families))
        #canvas.plot_point_sampling(sym_points, color=color)  
    all_points = np.vstack(all_points)
    all_values = np.vstack(all_values)
    
    canvas.plot_point_sampling_weighted(all_points, all_values)        
    canvas.plot_symmetry_cone(kspace)    
    
    regular_sampling_kx = np.arange(0, k0*1.1+max_lengths[0], max_lengths[0])
    regular_sampling_kx = np.concatenate([-regular_sampling_kx[:0:-1], regular_sampling_kx])
    
    regular_sampling_ky = np.arange(0, k0*1.1+max_lengths[1], max_lengths[1])
    regular_sampling_ky = np.concatenate([-regular_sampling_ky[:0:-1], regular_sampling_ky])
    
    KX, KY = np.meshgrid(regular_sampling_kx, regular_sampling_ky, indexing='ij')    
    Z = np.zeros(KX.shape)    
    
    #interp = LinearNDInterpolator(all_points[:,:2], all_values, fill_value=0.)
    interp = CloughTocher2DInterpolator(all_points[:,:2], all_values, fill_value=0.)
    
    grid = np.vstack([KX.flatten(), KY.flatten()]).T
    print(all_points.shape)
    print(grid.shape)
    
    
    #for ix in range(KX.shape[0]):
    #    for iy in range(KX.shape[1]):
    #        kx = KX[ix, iy]
    #        ky = KY[ix, iy]
            #index = np.where( np.logical_and(np.isclose(all_points[:,0], kx), np.isclose(all_points[:,1], ky)))            
            #print(index)
            #print(len(index))
            #if len(index[0]) > 0:
            #    Z[ix, iy] = all_values[index[0]]
    Z = interp(grid).reshape(KX.shape)
    
    plt.pcolormesh(KX, KY, Z, shading='gouraud')
    #interp = RectBivariateSpline(all_points[:,0], all_points[:,1], )
    
    print(max_lengths)
    print(regular_sampling_kx)
    print(regular_sampling_ky)
    
    print(KX.shape)
    #print(all_points.shape)
    #print(all_values.shape)    
    #print("int element: {}".format(int_element))
    #print(np.sum(all_values))    
    #print(int_element*np.sum(all_values))
    #print(np.pi*k0**2)
    #print("estimate: {}".format(int_element*np.sum(all_values)/(np.pi*k0**2)))
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim([-k0*1.1, k0*1.1])
    plt.ylim([-1.1*k0, 1.1*k0])
    plt.title(lat_shape, fontsize=20)

# Real Space Lattices

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    canvas.plot_tesselation(lat)
    canvas.plot_lattice(lat)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim([-2, 2])
    plt.ylim([-2, 2])
    plt.title(lat_shape, fontsize=20)    

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    canvas.plot_tesselation(lat)
    #canvas.plot_lattice(lat)    
    sampling, weighting, int_element = lat.unit_cell.sample(use_symmetry=False)
    canvas.plot_point_sampling(sampling)
    print(np.unique(weighting))
    print(np.sum(weighting*int_element)/lat.unit_cell.area())
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim([-2, 2])
    plt.ylim([-2, 2])
    plt.title(lat_shape, fontsize=20)

# Reciprocal Lattices
## Irreducible Brillouin Zone & High Symmetry Points

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    canvas.plot_tesselation(rlat)    
    canvas.plot_vectors(rlat)    
    canvas.plot_irreducible_uc(rlat.unit_cell)
    canvas.plot_special_points(rlat.unit_cell)
    

    plt.xlabel('kx')
    plt.ylabel('ky')
    plt.xlim([-10, 10])
    plt.ylim([-10, 10])
    plt.title(lat_shape, fontsize=20)

## Weighted Sampling of the BZ

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    #print(lat_shape)
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    #print("lattice lengths: [{} {}]".format(rlat.vectors.length1, rlat.vectors.length2))
    canvas.plot_tesselation(rlat)    
    constraint = {'type':'n_points', 'value':5}
    sampling, weighting, int_element = rlat.unit_cell.sample(use_symmetry=False,
                                                            constraint=constraint)
    #print(sampling)
    #print("integration element: {}".format(int_element))
    print("unique weighting: {}".format(np.unique(weighting)))
    #print(np.sum(weighting))
    #print(weighting)
    canvas.plot_point_sampling_weighted(sampling, weighting)
    full_area = rlat.unit_cell.area()
    weight_sum = np.sum(int_element*weighting)
    print(rlat.unit_cell.area())
    print("integration value check: {}".format(weight_sum/rlat.unit_cell.area()))
    #print("simple check: {}".format(full_area/int_element))
    plt.xlabel('kx')
    plt.ylabel('ky')
    plt.xlim([-5, 5])
    plt.ylim([-5, 5])
    plt.title(lat_shape, fontsize=20)

## Weighted Sampling of the IBZ

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    print(lat_shape)
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    canvas.plot_tesselation(rlat)    
    sampling, weighting, int_element, sym_ops = rlat.unit_cell.sample_irreducible()
    
    print("integration element: {}".format(int_element))
    print("unique weighting: {}".format(np.unique(weighting)))
    canvas.plot_point_sampling_weighted(sampling, weighting)
    full_area = rlat.unit_cell.area()
    weight_sum = np.sum(int_element*weighting)
    symmetry_multiplier = rlat.unit_cell.symmetry().get_n_symmetry_ops()
    print("integration value check: {}".format(symmetry_multiplier*weight_sum/full_area))
    plt.xlabel('kx')
    plt.ylabel('ky')
    plt.xlim([-5, 5])
    plt.ylim([-5, 5])
    plt.title(lat_shape, fontsize=20)

## BZ Sampling from symmetry

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    print(lat_shape)
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    #print("lattice lengths: [{} {}]".format(rlat.vectors.length1, rlat.vectors.length2))
    canvas.plot_tesselation(rlat)    
    canvas.plot_vectors(rlat)  
    constraint = {'type':'n_points', 'value':4}
    sampling, weighting, int_element = rlat.unit_cell.sample(use_symmetry=True,
                                                            constraint=constraint)
    #print(sampling)
    #print("integration element: {}".format(int_element))
    #print("unique weighting: {}".format(np.unique(weighting)))
    #print(np.sum(weighting))
    canvas.plot_point_sampling_weighted(sampling, weighting)
    #full_area = rlat.unit_cell.area()
    weight_sum = np.sum(weighting)
    print("integration value check: {}".format(weight_sum))
    #print("simple check: {}".format(full_area/int_element))
    plt.xlabel('kx')
    plt.ylabel('ky')
    plt.xlim([-7, 7])
    plt.ylim([-7, 7])
    plt.title(lat_shape, fontsize=20)

# Kspace
## 

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    print(lat_shape)
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])
    canvas = Canvas(ax=ax)
    wvl = 1.75
    k0 = 2*np.pi/wvl
    kspace = KSpace(wvl, symmetry=get_symmetry(lat_shape), fermi_radius=k0)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    kspace.apply_lattice(rlat)
    canvas.plot_tesselation(rlat)            
    canvas.plot_fermi_circle(kspace)
    #families = kspace.periodic_sampler.sample_bloch_families(restrict_to_sym_cone=True)
    constraint = {'type':'n_points', 'value':3}
    sampling, weighting, int_element, sym_ops = rlat.unit_cell.sample_irreducible(constraint=constraint)
    plt.scatter(sampling[:, 0], sampling[:,1], s=300, c='tab:red', label='IBZ Sampling')
    #print(sampling.shape)
    #print(len(sym_ops))
    #print(len(sym_ops[0]))
    #canvas.plot_point_sampling(sampling)
    families, all_kv, sym_ops = kspace.periodic_sampler._bloch_fam_from_sample(sampling, 1e-5, False, symmetries=sym_ops)
    #canvas.plot_bloch_families(families)
    #print("n families: {}".format(len(families)))
    all_points = []
    for num, fam in families.items():
        #if not num == 1:
        #    continue
        #canvas.plot_point_sampling(fam, color=color)        
        #plt.scatter(fam.kx, fam.ky, s=200, c='k')
        #print(fam.kx, fam.ky, fam.order1, fam.order2, sym_ops[num])
        #for row in range(fam.n_rows):
        #print(sym_ops[num])
        #print(sym_ops[num])
        zeroth_order = np.where(np.logical_and(fam.order1==0, fam.order2==0))
        #print(fam.kx[zeroth_order], fam.ky[zeroth_order], sym_ops[num])
        sym = rlat.unit_cell.symmetry()-sym_ops[num]
        
        #print(fam.k.shape)
        sym_points = sym.apply_symmetry_operators(fam.k)
        #print("sym points shape: {}".format(sym_points.shape))
        sym_points = np.unique(sym_points.round(decimals=4), axis=0)
        #print("sym points shape after unique: {}".format(sym_points.shape))
        #print(fam.n_rows, sym_points.shape[0])
        #print(sym_points)
        all_points.append(sym_points)
        #plt.scatter(sym_points[:,0], sym_points[:,1], s=50, c='tab:blue')
        #print(sym)
        #print(sym_points)
        #print(sym_points.shape[0])        
        #color = choose_color(num, len(families))
        #canvas.plot_point_sampling(sym_points, color=color)    
    all_points = np.vstack(all_points)
    plt.scatter(all_points[:,0], all_points[:, 1], s=200, c='k', label="BF Sampling")
    
    if True:
        duplicates = []
        is_duplicate = np.zeros(all_points.shape[0], dtype=bool)
        for row in range(all_points.shape[0]):
            point0 = all_points[row, :]
            if is_duplicate[row]:
                pass #continue
            for row2 in range(all_points.shape[0]):
                if row == row2:
                    continue
                if is_duplicate[row2]:
                    pass #continue
                point1 = all_points[row2, :]
                if np.linalg.norm(point0-point1) < 1e-6:
                    duplicates.append(point0)
                    is_duplicate[row] = True
                    is_duplicate[row2] = True
        if len(duplicates) > 0:
            duplicates = np.vstack(duplicates)
            plt.scatter(duplicates[:, 0], duplicates[:, 1], s=50, c='tab:blue', label="Duplicates")
    canvas.plot_symmetry_cone(kspace)
    plt.legend(bbox_to_anchor=[0., 1.1])
    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim([-k0*1.5, k0*1.5])
    plt.ylim([-1.5*k0, 1.5*k0])
    plt.title(lat_shape, fontsize=20)

In [ ]:
def lorentz(gamma, x0, x):
    return (1./np.pi) * 0.5*gamma / ( (x-x0)**2 + (0.5*gamma)**2)

def int_lorentz(gamma, x0, x):
    return (1./np.pi)*np.arctan(2*(x-x0)/gamma)

def int_lorentz_radial(gamma, x0, x):
    return (1./np.pi)*(gamma*np.log( 4*(x-x0)**2 + gamma**2)/4.0 + x0 * np.arctan( 2*(x-x0)/gamma))

def cusp_function(rho, width, pos):
    L = lorentz(width, pos, rho)
    return L
    
def int_cusp_function(rho):
    L_int = int_lorentz(0.05, 0.5, rho)
    return L_int + (10./4.)*rho**4 + 10*rho

def int_cusp_function_radial(rho):
    L_int_radial = int_lorentz_radial(0.05, 0.5, rho)
    return L_int_radial + (10./5.)*rho**5 + (10/2.)*rho**2

def radial_cusp_function(kx, ky, kmax):
    kr = np.sqrt( kx**2 + ky**2)/kmax    
    pos = kmax*0.4
    width = kmax/20
    return cusp_function(kr, width, pos)

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    print(lat_shape)
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])
    canvas = Canvas(ax=ax)
    wvl = 1.5*np.pi
    k0 = 2*np.pi/wvl
    kspace = KSpace(wvl, symmetry=get_symmetry(lat_shape), fermi_radius=k0)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    kspace.apply_lattice(rlat)
    canvas.plot_tesselation(rlat)            
    canvas.plot_fermi_circle(kspace)
    #families = kspace.periodic_sampler.sample_bloch_families(restrict_to_sym_cone=True)
    constraint = {'type':'n_points', 'value':5}
    sampling, weighting, int_element, sym_ops = rlat.unit_cell.sample_irreducible(constraint=constraint)
    #plt.scatter(sampling[:, 0], sampling[:,1], s=300, c='tab:red')
    #print(sampling.shape)
    #f_vals = radial_cusp_function(sampling[:, 0], sampling[:,1], k0)
    #canvas.plot_point_sampling_weighted(sampling, f_vals)
    #print(len(sym_ops))
    #print(len(sym_ops[0]))
    #canvas.plot_point_sampling(sampling)
    families, all_kv, sym_ops = kspace.periodic_sampler._bloch_fam_from_sample(sampling, 1e-5, False, symmetries=sym_ops)
    #canvas.plot_bloch_families(families)
    all_points = []
    all_values = []
    for num, fam in families.items():
        #if not num == 1:
        #    continue
        #color = choose_color(num, len(families))
        
        kx = fam.kx
        ky = fam.ky        
        f_vals = radial_cusp_function(fam.kx, fam.ky, k0)
        f_vals = np.atleast_2d(f_vals).T
        #print(f_vals.shape)
        kxy = np.vstack([kx, ky]).T
        #print(kxy.shape)
        print_data = np.hstack([kxy, f_vals])
        #print(print_data.shape)
        #print(print_data)
        #canvas.plot_point_sampling_weighted(fam.k[:, :2], f_vals)
        #plt.scatter(fam.kx, fam.ky, s=200, c='k')
        #print(fam.kx, fam.ky, fam.order1, fam.order2, sym_ops[num])
        #for row in range(fam.n_rows):
        #print(sym_ops[num])
        #print(sym_ops[num])
        zeroth_order = np.where(np.logical_and(fam.order1==0, fam.order2==0))
        #print(fam.kx[zeroth_order], fam.ky[zeroth_order], sym_ops[num])
        sym = rlat.unit_cell.symmetry()-sym_ops[num]
        #print(sym)
        #print(fam.k.shape, f_vals.shape)        
        sym_points, sym_vals = sym.apply_symmetry_operators(fam.k, values=f_vals)
        #print(sym_points.shape, sym_vals.shape)
        sym_points, indices = np.unique(sym_points.round(decimals=4), axis=0, return_index=True)
        sym_vals = sym_vals[indices, :]
        #print(fam.n_rows, sym_points.shape[0])
        #print(sym_points)
        all_points.append(sym_points)
        all_values.append(sym_vals)
        #plt.scatter(sym_points[:,0], sym_points[:,1], s=50, c='tab:blue')
        #print(sym)
        #print(sym_points)
        #print(sym_points.shape[0])        
        #color = choose_color(num, len(families))
        #canvas.plot_point_sampling(sym_points, color=color)  
    all_points = np.vstack(all_points)
    all_values = np.vstack(all_values)
    canvas.plot_point_sampling_weighted(all_points, all_values)        
    canvas.plot_symmetry_cone(kspace)
    
    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim([-k0*1.5, k0*1.5])
    plt.ylim([-1.5*k0, 1.5*k0])
    plt.title(lat_shape, fontsize=20)

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    canvas.plot_tesselation(rlat)
    canvas.plot_lattice(rlat)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim([-10, 10])
    plt.ylim([-10, 10])
    plt.title(lat_shape, fontsize=20)

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    canvas.plot_tesselation(rlat)        
    sample = rlat.unit_cell.sample()[0]
    canvas.plot_sampling(sample)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim([-10, 10])
    plt.ylim([-10, 10])
    plt.title(lat_shape, fontsize=20)

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    wvl = 0.5
    k0 = 2*np.pi/wvl
    kspace = KSpace(wvl, symmetry=get_symmetry(lat_shape), fermi_radius=k0)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    kspace.apply_lattice(rlat)
    canvas.plot_tesselation(rlat)            
    canvas.plot_fermi_circle(kspace)
    families, all_points = kspace.periodic_sampler.sample_bloch_families()
    canvas.plot_bloch_families(families)
      
    
    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim([-15, 15])
    plt.ylim([-15, 15])
    plt.title(lat_shape, fontsize=20)

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    wvl = 0.5
    k0 = 2*np.pi/wvl
    kspace = KSpace(wvl, symmetry=get_symmetry(lat_shape), fermi_radius=k0)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    kspace.apply_lattice(rlat)
    canvas.plot_tesselation(rlat)            
    canvas.plot_fermi_circle(kspace)
    families = kspace.periodic_sampler.sample_bloch_families(restrict_to_sym_cone=True)
    canvas.plot_bloch_families(families)
    canvas.plot_symmetry_cone(kspace)
    
    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim([-15, 15])
    plt.ylim([-15, 15])
    plt.title(lat_shape, fontsize=20)

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    wvl = 0.5
    k0 = 2*np.pi/wvl
    kspace = KSpace(wvl, symmetry=get_symmetry(lat_shape), fermi_radius=k0)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    kspace.apply_lattice(rlat)
    canvas.plot_tesselation(rlat)
    canvas.plot_fermi_circle(kspace)
    sampling = kspace.periodic_sampler.sample(restrict_to_sym_cone=True, constraint={'type':'n_points', 'value':4})
    #canvas.plot_sampling(sampling.k,  color='k')
    groups = kspace.symmetrise_sample(sampling)
    canvas.plot_symmetry_cone(kspace)    
    
    cmap = cm.get_cmap('tab20')
    NColors = 20
    for i in range(len(groups)):
        color = np.zeros((1,4))
        color[0,:] = cmap((float(i))/NColors)
        canvas.plot_point_sampling(groups[i], plot_n_points='all', color = color)
    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim([-15, 15])
    plt.ylim([-15, 15])
    plt.title(lat_shape, fontsize=20)

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    wvl = 2.0
    k0 = 2*np.pi*wvl
    kspace = KSpace(wvl, symmetry=get_symmetry(lat_shape), fermi_radius=k0)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    kspace.apply_lattice(rlat)
    canvas.plot_tesselation(rlat)            
    canvas.plot_fermi_circle(kspace)
    woods1 = kspace.periodic_sampler.calc_woods_anomalies(1, n_refinements = 4)
    cmap = cm.get_cmap('tab20')
    NColors = 20
    for i in range(len(woods1)):
        color = np.zeros((1,4))
        color[0,:] = cmap((float(i))/NColors)    
        canvas.plot_point_sampling(woods1[i], plot_n_points='all', color =color)

    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim([-15, 15])
    plt.ylim([-15, 15])
    plt.title(lat_shape, fontsize=20)

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    wvl = 2.0
    k0 = 2*np.pi*wvl
    kspace = KSpace(wvl, symmetry=get_symmetry(lat_shape), fermi_radius=k0)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    kspace.apply_lattice(rlat)
    canvas.plot_tesselation(rlat)            
    canvas.plot_fermi_circle(kspace)
    woods1 = kspace.periodic_sampler.calc_woods_anomalies(2, n_refinements = 4)
    cmap = cm.get_cmap('tab20')
    NColors = 20
    for i in range(len(woods1)):
        color = np.zeros((1,4))
        color[0,:] = cmap((float(i))/NColors)    
        canvas.plot_point_sampling(woods1[i], plot_n_points='all', color =color)

    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim([-15, 15])
    plt.ylim([-15, 15])
    plt.title(lat_shape, fontsize=20)

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    wvl = 2.0
    k0 = 2*np.pi*wvl
    kspace = KSpace(wvl, symmetry=get_symmetry(lat_shape), fermi_radius=k0)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    kspace.apply_lattice(rlat)
    canvas.plot_tesselation(rlat)            
    canvas.plot_fermi_circle(kspace)
    woods1 = kspace.periodic_sampler.calc_woods_anomalies(1, n_refinements = 4, restrict_to_sym_cone=True)
    cmap = cm.get_cmap('tab20')
    NColors = 20
    for i in range(len(woods1)):
        color = np.zeros((1,4))
        color[0,:] = cmap((float(i))/NColors)    
        canvas.plot_point_sampling(woods1[i], plot_n_points='all', color =color)
    canvas.plot_symmetry_cone(kspace)
    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim([-15, 15])
    plt.ylim([-15, 15])
    plt.title(lat_shape, fontsize=20)

In [ ]:
fig = plt.figure(figsize=(12,12))
gs = gridspec.GridSpec(2, 2, fig, wspace=0.4, hspace=0.4)

for ilat, lat_shape in enumerate(['square', 'rectangle', 'hexagon', 'oblique']):
    index =  np.unravel_index(ilat, (2,2))
    ax = fig.add_subplot(gs[index])    
    canvas = Canvas(ax=ax)
    wvl = 2.0
    k0 = 2*np.pi*wvl
    kspace = KSpace(wvl, symmetry=get_symmetry(lat_shape), fermi_radius=k0)
    lat_vec = get_lattice_vectors(lat_shape)
    lat = Lattice(lat_vec)
    rlat = lat.make_reciprocal()
    kspace.apply_lattice(rlat)
    canvas.plot_tesselation(rlat)            
    canvas.plot_fermi_circle(kspace)
    woods1 = kspace.periodic_sampler.calc_woods_anomalies(1, n_refinements = 4, restrict_to_sym_cone=True)
    cmap = cm.get_cmap('tab20')
    NColors = 20
    for i in range(len(woods1)):
        color = np.zeros((1,4))
        color[0,:] = cmap((float(i))/NColors)    
        #canvas.plot_point_sampling(woods1[i], plot_n_points='all', color =color)
        sym_woods = kspace.symmetrise_sample(woods1[i])
        for i_wood in range(len(sym_woods)):
            canvas.plot_point_sampling(sym_woods[i_wood], color=color)
        
    canvas.plot_symmetry_cone(kspace)
    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim([-15, 15])
    plt.ylim([-15, 15])
    plt.title(lat_shape, fontsize=20)